# Obtaining Data

In [34]:
#Importing libraries needed
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import pyplot
%matplotlib inline
import numpy as np
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [35]:
#opening the datasets 
one = pd.read_csv(r'data\oneyear.csv')
six =  pd.read_csv(r'data\sixmonths.csv')
three =  pd.read_csv(r'data\threemonths.csv')
top =  pd.read_csv(r'data\top.csv')
bottom =  pd.read_csv(r'data\bottom.csv')

## Logistic Regression on Six Months Before and After

In [36]:
six.head()

,City,DATE,AWND,WSF2,WSF5,HurricaneName,SizeRank,bc,ac,bd,...,percentc,percentd,percentma,percentir,percentmi,boolc,boold,boolma,boolir,boolmi
0,Apalachicola,8/14/2004,5.82,13.0,15.0,1,12877,91915.39335,114408.1491,112287.8968,...,24.471152,24.881677,3.115506,11.045477,11.724797,1,0,0,1,1
1,Apalachicola,7/10/2005,19.46,30.0,34.9,2,12877,91915.39335,114408.1491,112287.8968,...,24.471152,24.881677,3.115506,11.045477,11.724797,1,0,0,1,1
2,Apalachicola,7/11/2005,17.00,32.0,38.0,2,12877,91915.39335,114408.1491,112287.8968,...,24.471152,24.881677,3.115506,11.045477,11.724797,1,0,0,1,1
3,Apalachicola,10/7/2016,10.74,21.9,27.1,3,12877,91915.39335,114408.1491,112287.8968,...,24.471152,24.881677,3.115506,11.045477,11.724797,1,0,0,1,1
4,Apalachicola,10/8/2016,8.05,15.0,21.9,3,12877,91915.39335,114408.1491,112287.8968,...,24.471152,24.881677,3.115506,11.045477,11.724797,1,0,0,1,1


### Data Scrubbing

In [37]:
#let's drop any column we don't need for modeling 
six.drop('DATE', axis = 1, inplace=True)
six = six.loc[:, ~six.columns.str.startswith('a')]
six = six.loc[:, ~six.columns.str.startswith('percent')]
six

,City,AWND,WSF2,WSF5,HurricaneName,SizeRank,bc,bd,bma,bir,bmi,boolc,boold,boolma,boolir,boolmi
0,Apalachicola,5.82,13.0,15.0,1,12877,91915.39335,112287.8968,124938.9556,127923.4267,144565.8722,1,0,0,1,1
1,Apalachicola,19.46,30.0,34.9,2,12877,91915.39335,112287.8968,124938.9556,127923.4267,144565.8722,1,0,0,1,1
2,Apalachicola,17.00,32.0,38.0,2,12877,91915.39335,112287.8968,124938.9556,127923.4267,144565.8722,1,0,0,1,1
3,Apalachicola,10.74,21.9,27.1,3,12877,91915.39335,112287.8968,124938.9556,127923.4267,144565.8722,1,0,0,1,1
4,Apalachicola,8.05,15.0,21.9,3,12877,91915.39335,112287.8968,124938.9556,127923.4267,144565.8722,1,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
388,West Palm Beach,38.92,66.0,91.0,4,158,156234.53760,187824.4966,197641.9759,214427.8925,235358.1321,1,0,0,0,0
389,West Palm Beach,20.80,38.0,49.0,4,158,156234.53760,187824.4966,197641.9759,214427.8925,235358.1321,1,0,0,0,0
390,West Palm Beach,6.93,17.0,21.9,5,158,156234.53760,187824.4966,197641.9759,214427.8925,235358.1321,1,0,0,0,0
391,West Palm Beach,12.75,25.1,33.1,5,158,156234.53760,187824.4966,197641.9759,214427.8925,235358.1321,1,0,0,0,0


### Feature Engineering 
We will be grouping the values by max wind speed

In [38]:
six = six.groupby(['HurricaneName', 'City']).max()
six

AWND  WSF2  WSF5  SizeRank            bc  \
HurricaneName City                                                         
1             Apalachicola      5.82  13.0  15.0     12877   91915.39335   
              Brooksville       7.38  21.9  31.1      1410  118348.26420   
              Clearwater        9.62  25.1  32.0       233  133267.09810   
              Cross City        4.47  14.1  17.9     11954   38971.28538   
              Daytona Beach     8.28   8.1   8.1       744  107321.80230   
...                              ...   ...   ...       ...           ...   
5             Tallahassee      23.71  49.0  70.9        89  142481.87370   
              Tampa            14.99  25.1  35.1        50  134130.40310   
              Tavares          13.87  35.1  49.0      2776  122114.20660   
              Vero Beach       14.54  21.9  28.0       350  148874.73750   
              West Palm Beach  13.65  25.1  33.1       158  156234.53760   

                                         bd           bma          bir  \
HurricaneName City                                                       
1             Apalachicola     112287.89680  124938.95560  127923.4267   
              Brooksville      136807.94300  131341.02820  145960.5536   
              Clearwater       153662.21160  164004.34390  179891.1530   
              Cross City        47433.41348   55644.25987   55676.5735   
              Daytona Beach    129664.06090  119402.40490  131778.2043   
...                                     ...           ...          ...   
5             Tallahassee      160567.08110  160448.93870  168065.0822   
              Tampa            156105.36820  175969.64650  198490.9291   
              Tavares          140538.67340  152285.65450  166801.3463   
              Vero Beach       179526.51710  175538.38400  192699.9478   
              West Palm Beach  187824.49660  197641.97590  214427.8925   

                                        bmi  boolc  boold  boolma  boolir  \
HurricaneName City                                                          
1             Apalachicola     144565.87220      1      0       0       1   
              Brooksville      162901.00290      0      1       1       0   
              Clearwater       199513.87270      0      0       1       0   
              Cross City        58194.97041      1      0       0       0   
              Daytona Beach    149756.03060      1      1       1       1   
...                                     ...    ...    ...     ...     ...   
5             Tallahassee      178534.22650      0      0       0       0   
              Tampa            222385.57760      0      0       1       1   
              Tavares          184998.99610      0      1       1       0   
              Vero Beach       214829.58660      1      1       1       0   
              West Palm Beach  235358.13210      1      0       0       0   

                               boolmi  
HurricaneName City                     
1             Apalachicola          1  
              Brooksville           0  
              Clearwater            0  
              Cross City            1  
              Daytona Beach         1  
...                               ...  
5             Tallahassee           0  
              Tampa                 0  
              Tavares               0  
              Vero Beach            0  
              West Palm Beach       0  

[139 rows x 14 columns]

In [40]:
six.reset_index(inplace = True)
six.head()

,HurricaneName,City,AWND,WSF2,WSF5,SizeRank,bc,bd,bma,bir,bmi,boolc,boold,boolma,boolir,boolmi
0,1,Apalachicola,5.82,13.0,15.0,12877,91915.39335,112287.89680,124938.95560,127923.4267,144565.87220,1,0,0,1,1
1,1,Brooksville,7.38,21.9,31.1,1410,118348.26420,136807.94300,131341.02820,145960.5536,162901.00290,0,1,1,0,0
2,1,Clearwater,9.62,25.1,32.0,233,133267.09810,153662.21160,164004.34390,179891.1530,199513.87270,0,0,1,0,0
3,1,Cross City,4.47,14.1,17.9,11954,38971.28538,47433.41348,55644.25987,55676.5735,58194.97041,1,0,0,0,1
4,1,Daytona Beach,8.28,8.1,8.1,744,107321.80230,129664.06090,119402.40490,131778.2043,149756.03060,1,1,1,1,1


In [41]:
#making a dataframe for each hurricane
sixc = six[['HurricaneName', 'City', 'AWND', 'WSF2', 'WSF5', 'SizeRank', 'bc', 'boolc']]
sixd = six[['HurricaneName', 'City', 'AWND', 'WSF2', 'WSF5', 'SizeRank', 'bd', 'boold']]
sixma = six[['HurricaneName', 'City', 'AWND', 'WSF2', 'WSF5', 'SizeRank', 'bma', 'boolma']]
sixir = six[['HurricaneName', 'City', 'AWND', 'WSF2', 'WSF5', 'SizeRank', 'bir', 'boolir']]
sixmi = six[['HurricaneName', 'City', 'AWND', 'WSF2', 'WSF5', 'SizeRank', 'bmi', 'boolmi']]

In [44]:
sixc.head()

,HurricaneName,City,AWND,WSF2,WSF5,SizeRank,bc,boolc
0,1,Apalachicola,5.82,13.0,15.0,12877,91915.39335,1
1,1,Brooksville,7.38,21.9,31.1,1410,118348.26420,0
2,1,Clearwater,9.62,25.1,32.0,233,133267.09810,0
3,1,Cross City,4.47,14.1,17.9,11954,38971.28538,1
4,1,Daytona Beach,8.28,8.1,8.1,744,107321.80230,1


In [211]:
#check crosstabs 
df[df.Increase == 0].describe()

,AWND,WSF2,WSF5,SizeRank,Before,After,Percent,Increase
count,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.0
mean,19.563000,35.890000,48.965714,543.257143,207454.285714,213262.857143,2.818421,0.0
std,6.912737,13.285376,17.896910,731.204797,78722.879730,81678.172224,4.541435,0.0
min,10.510000,18.100000,23.000000,13.000000,109500.000000,117200.000000,-12.659123,0.0
25%,14.375000,25.300000,36.000000,89.250000,152125.000000,157850.000000,0.140577,0.0
50%,17.670000,31.100000,42.100000,291.000000,188350.000000,192950.000000,3.716528,0.0
75%,21.920000,45.300000,66.325000,655.750000,245275.000000,253525.000000,5.822047,0.0
max,40.710000,70.900000,87.900000,2830.000000,614400.000000,643100.000000,9.716387,0.0


In [212]:
df[df.Increase == 1].describe()

,AWND,WSF2,WSF5,SizeRank,Before,After,Percent,Increase
count,25.000000,25.000000,25.000000,25.00000,25.000000,25.000000,25.000000,25.0
mean,21.555200,39.052000,54.000000,509.52000,172364.000000,196208.000000,13.805276,1.0
std,9.163215,15.391126,20.807611,606.77303,77081.882221,90191.430302,3.798559,0.0
min,8.950000,19.900000,31.100000,13.00000,111700.000000,128500.000000,10.038860,1.0
25%,14.320000,25.900000,33.100000,127.00000,138900.000000,160600.000000,11.916376,1.0
50%,19.240000,33.100000,47.000000,340.00000,154400.000000,172800.000000,12.557870,1.0
75%,25.500000,52.100000,69.100000,577.00000,172900.000000,197900.000000,14.459225,1.0
max,40.260000,74.000000,91.000000,2390.00000,510100.000000,597300.000000,25.937261,1.0


# Logistic Regression

In [42]:
# For our modeling steps
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import log_loss

# For demonstrative pruposes
from scipy.special import logit, expit

In [43]:
logreg = LogisticRegression(random_state=56)

### Modeling Six Months Before and After Charley First

In [46]:
sixc.head()

,HurricaneName,City,AWND,WSF2,WSF5,SizeRank,bc,boolc
0,1,Apalachicola,5.82,13.0,15.0,12877,91915.39335,1
1,1,Brooksville,7.38,21.9,31.1,1410,118348.26420,0
2,1,Clearwater,9.62,25.1,32.0,233,133267.09810,0
3,1,Cross City,4.47,14.1,17.9,11954,38971.28538,1
4,1,Daytona Beach,8.28,8.1,8.1,744,107321.80230,1


In [52]:
#y is prediction variable 
#X is features 
y = sixc['boolc']
X = sixc.drop(['boolc', 'City'], axis=1)

In [58]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=56)

In [59]:
logreg.fit(X_train, y_train)

LogisticRegression(random_state=56)

In [60]:
logreg.predict(X_train)

array([0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [61]:
logreg.predict(X_test)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
      dtype=int64)